## Import and load the data file

In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
# The pickle module is used for serializing(saving) and deserializing(loading) Python objects.

In [2]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
#An optimizer adjusts the weights of the neurons to minimize the error in predictions during training.
from keras.optimizers import SGD
import random

In [3]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intent.json').read()
intents = json.loads(data_file)

## Preprocess data

In [4]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [5]:
# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)


pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

80 documents
21 classes ['age', 'books', 'food', 'goodbye', 'greeting', 'happy', 'hate', 'help', 'joke', 'laugh', 'love', 'movies', 'music', 'name', 'news', 'options', 'sad', 'story', 'thanks', 'time', 'weather']
108 unique lemmatized words ["'m", "'s", ',', 'a', 'age', 'am', 'any', 'anyone', 'are', 'assistance', 'awesome', 'be', 'book', 'bored', 'bye', 'call', 'can', 'chatting', 'cool', 'could', 'current', 'day', 'do', 'down', 'eat', 'everyone', 'for', 'forecast', 'fun', 'funny', 'good', 'goodbye', 'great', 'ha', 'haha', 'happening', 'happy', 'hate', 'hehe', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'hola', 'how', 'hungry', 'i', 'in', 'is', 'it', 'joke', 'know', 'later', 'latest', 'like', 'listen', 'lol', 'love', 'may', 'me', 'movie', 'music', 'myself', 'name', 'need', 'news', 'next', 'nice', 'offered', 'old', 'play', 'please', 'provide', 'read', 'recommend', 'restaurant', 'sad', 'see', 'should', 'some', 'story', 'suggest', 'suggestion', 'sunny', 'support', 'tell', 'thank', 

## Create training and testing data

In [6]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


C:\Users\win-10\AppData\Local\Temp\ipykernel_18472\3528693901.py:24: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


## Build the model 

In [7]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))  
#Adds a Dropout layer with a dropout rate of 0.5, meaning 50% of the neurons are randomly dropped during training to prevent overfitting
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

C:\Users\win-10\anaconda3\lib\site-packages\keras\optimizers\legacy\gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/200
16/16 [==============================] - 1s 3ms/step - loss: 3.0535 - accuracy: 0.0375 
Epoch 2/200
16/16 [==============================] - 0s 3ms/step - loss: 3.0535 - accuracy: 0.0625
Epoch 3/200
16/16 [==============================] - 0s 4ms/step - loss: 2.9574 - accuracy: 0.1500
Epoch 4/200
16/16 [==============================] - 0s 3ms/step - loss: 2.9075 - accuracy: 0.1625
Epoch 5/200
16/16 [==============================] - 0s 3ms/step - loss: 2.8594 - accuracy: 0.1375
Epoch 6/200
16/16 [==============================] - 0s 3ms/step - loss: 2.8450 - accuracy: 0.2125
Epoch 7/200
16/16 [==============================] - 0s 4ms/step - loss: 2.7633 - accuracy: 0.1500
Epoch 8/200
16/16 [==============================] - 0s 3ms/step - loss: 2.6626 - accuracy: 0.2625
Epoch 9/200
16/16 [==============================] - 0s 4ms/step - loss: 2.6238 - accuracy: 0.2250
Epoch 10/200
16/16 [==============================] - 0s 3ms/step - loss: 2.5170 - accuracy: 0.2250
Epoch 11

16/16 [==============================] - 0s 3ms/step - loss: 0.1434 - accuracy: 0.9500
Epoch 84/200
16/16 [==============================] - 0s 3ms/step - loss: 0.1964 - accuracy: 0.9625
Epoch 85/200
16/16 [==============================] - 0s 2ms/step - loss: 0.0871 - accuracy: 1.0000
Epoch 86/200
16/16 [==============================] - 0s 3ms/step - loss: 0.1642 - accuracy: 0.9750
Epoch 87/200
16/16 [==============================] - 0s 2ms/step - loss: 0.1798 - accuracy: 0.9875
Epoch 88/200
16/16 [==============================] - 0s 2ms/step - loss: 0.1928 - accuracy: 0.9375
Epoch 89/200
16/16 [==============================] - 0s 3ms/step - loss: 0.2029 - accuracy: 0.9500
Epoch 90/200
16/16 [==============================] - 0s 3ms/step - loss: 0.2357 - accuracy: 0.9250
Epoch 91/200
16/16 [==============================] - 0s 3ms/step - loss: 0.1803 - accuracy: 0.9625
Epoch 92/200
16/16 [==============================] - 0s 3ms/step - loss: 0.2498 - accuracy: 0.8875
Epoch 93/200
